In [1]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [2]:
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
model = IALSRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])
earlystopping_kwargs =  {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": "MAP",
                              }

EvaluatorHoldout: Ignoring 2628 (20.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [3]:
import optuna as op
def objective(trial):
    num_factors = trial.suggest_int("num_factors", 1, 1000)
    epochs = 300
    confidence_scaling = trial.suggest_categorical("confidence_scaling", ["linear", "log"])
    alpha = trial.suggest_loguniform("alpha", 1e-10, 10.0)
    epsilon = trial.suggest_loguniform("epsilon", 1e-10, 10.0)
    reg = trial.suggest_loguniform("reg", 1e-10, 10.0)
    
    recommender = model(URM_train)
    recommender.fit(**trial.params, **earlystopping_kwargs)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    "num_factors": 91,
    "epochs": 50,
    "confidence_scaling": "log",
    "alpha": 8.931839513881036,
    "epsilon": 6.099966408152877,
    "reg": 0.0003668646666587094,
}

# {"num_factors": 91, "epochs": 50, "confidence_scaling": "log", "alpha": 8.931839513881036, "epsilon": 6.099966408152877, "reg": 0.0003668646666587094}
study_name = "IALS-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize")
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=30)

In [9]:
study.best_params

{'topK': 101, 'alpha': 0.3026342852596128, 'beta': 0.058468783118329024}

In [10]:
final = model(URM_train_validation)
final.fit(**study.best_params)

RP3betaRecommender: URM Detected 245 ( 1.9%) users with no interactions.
RP3betaRecommender: URM Detected 110 ( 0.5%) items with no interactions.
RP3betaRecommender: Similarity column 22222 (100.0%), 4027.41 column/sec. Elapsed time 5.52 sec


In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2100 (16.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10538 (100.0%) in 5.18 sec. Users per second: 2033


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.090378                 0.157221  0.133194  0.045665    0.078973   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.259414  0.143182  0.107686  0.509869      0.339649  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.833834          0.425146    0.833834       0.057877   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10           10.145296                   0.997617             0.165954   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   